In [64]:
import pandas as pd
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
import numpy as np

In [81]:
weather = pd.read_csv('weather_cleaned_v1.csv')

In [82]:
weather.head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,Daylight
0,1,2007-05-01,83,50,67.0,14,51,56,0,2,0.0,29.10,29.82,1.7,27,9.2,14
1,2,2007-05-01,84,52,68.0,999,51,57,0,3,0.0,29.18,29.82,2.7,25,9.6,14
2,1,2007-05-02,59,42,51.0,-3,42,47,14,0,0.0,29.38,30.09,13.0,4,13.4,14
3,2,2007-05-02,60,43,52.0,999,42,47,13,0,0.0,29.44,30.08,13.3,2,13.4,14
4,1,2007-05-03,66,46,56.0,2,40,48,9,0,0.0,29.39,30.12,11.7,7,11.9,14


In [83]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2944 entries, 0 to 2943
Data columns (total 17 columns):
Station        2944 non-null int64
Date           2944 non-null object
Tmax           2944 non-null int64
Tmin           2944 non-null int64
Tavg           2944 non-null float64
Depart         2944 non-null int64
DewPoint       2944 non-null int64
WetBulb        2944 non-null int64
Heat           2944 non-null int64
Cool           2944 non-null int64
PrecipTotal    2944 non-null float64
StnPressure    2944 non-null float64
SeaLevel       2944 non-null float64
ResultSpeed    2944 non-null float64
ResultDir      2944 non-null int64
AvgSpeed       2944 non-null float64
Daylight       2944 non-null int64
dtypes: float64(6), int64(10), object(1)
memory usage: 391.1+ KB


In [84]:
weather["Date"] = pd.to_datetime(weather["Date"])

In [85]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2944 entries, 0 to 2943
Data columns (total 17 columns):
Station        2944 non-null int64
Date           2944 non-null datetime64[ns]
Tmax           2944 non-null int64
Tmin           2944 non-null int64
Tavg           2944 non-null float64
Depart         2944 non-null int64
DewPoint       2944 non-null int64
WetBulb        2944 non-null int64
Heat           2944 non-null int64
Cool           2944 non-null int64
PrecipTotal    2944 non-null float64
StnPressure    2944 non-null float64
SeaLevel       2944 non-null float64
ResultSpeed    2944 non-null float64
ResultDir      2944 non-null int64
AvgSpeed       2944 non-null float64
Daylight       2944 non-null int64
dtypes: datetime64[ns](1), float64(6), int64(10)
memory usage: 391.1 KB


In [86]:

def make_datetimeobject_col(df):
    new_df = df.copy()
    new_df['dtDate'] = pd.to_datetime(new_df['Date'])
    return new_df


def timelagged_daylight(obs_date):
    """
    Calcuates the hours of daylight 35 days prior to a given date.
    Formula is from Lebl et at, eqn. 1, modified by James Truslow because they screwed 
    up the daylength coefficient.
    
    Parameters
    ----------
    obs_date: a datetime timestamp for the observation
    
    
    Returns
    -------
    Number of daylight hours 35 days prior to 'obs_date'
    """
    
    import datetime as dt
    
    lag_days = 35
    time_lag = dt.timedelta(days=lag_days)
    lagged_date = obs_date - time_lag
    
    # This calculation of day of year will by off by leap years, and maybe an off-by-one for day 0
    day_of_year = (lagged_date.toordinal() - dt.datetime(2007,1,1).toordinal())%365   
    eps = np.arcsin(0.39795 * np.cos(0.2163108 + 2 * np.arctan(0.9671396 * np.tan(0.0086*(day_of_year - 186)))))
    phi = 41.983*2*np.pi/360
    D = 24 - (24/np.pi)*np.arccos((np.sin(0.8333*np.pi/180)+np.sin(phi)*np.sin(eps))/(np.cos(phi)*np.cos(eps)))
        
    return D


def make_timelagged_daylight_col(df):
    new_df = df.copy()
    new_df['TimeLaggedDaylight'] = new_df['dtDate'].map(timelagged_daylight)
    return new_df


def timelagged_temperature(df, obs_date, lag_days_LB=7, lag_days_UB=0):
    """
    Calculates the average temperature in Chicago in some time window before an observation
    
    Parameters
    ----------
    df: dataframe with weather info.  I hope there is date info as datetime objects
    obs_date: a datetime timestamp for the observation
    lag_days_LB: chronological lower bound of time window (inclusive), in number of days *before* observation
    lag_days_UB: chronological upper bound of time window (inclusive), in number of days *before* observation 
    
    Returns
    -------
    Average temperature in time window 
    """
    
    Date_obs = obs_date
    
    time_lag_LB = dt.timedelta(days=lag_days_LB)
    time_lag_UB = dt.timedelta(days=lag_days_UB)
    
    Date_LB = Date_obs - time_lag_LB
    Date_UB = Date_obs - time_lag_UB
    
    window_Tmean = df[(df.dtDate >= Date_LB) & (df.dtDate <= Date_UB)].avg_Tavg.mean()
    
    return window_Tmean


def make_timelagged_temperature_col(df):
    new_df = df.copy()
    new_df['TimeLaggedTemperature'] = df['dtDate'].map(lambda row: timelagged_temperature(df,row))
    return new_df


def timelagged_precipitation(df, obs_date, lag_days_LB=70, lag_days_UB=0):
    """
    Calcuates the average precipitation in Chicago in some time window before an observation
    
    Parameters
    ----------
    df: dataframe with weather info.  I hope there is date info as datetime objects
    obs_date: a datetime timestamp for the observation
    lag_days_LB: chronological lower bound of time window (inclusive), in number of days *before* observation
    lag_days_UB: chronological upper bound of time window (inclusive), in number of days *before* observation 
    
    Returns
    -------
    Average precipitation in time window 
    """
    
    Date_obs = obs_date
    
    time_lag_LB = dt.timedelta(days=lag_days_LB)
    time_lag_UB = dt.timedelta(days=lag_days_UB)
    
    Date_LB = Date_obs - time_lag_LB
    Date_UB = Date_obs - time_lag_UB
    
    window_Pmean = df[(df.dtDate >= Date_LB) & (df.dtDate <= Date_UB)].avg_PrecipTotal.mean()
    
    return window_Pmean


def make_timelagged_precipitation_col(df):
    new_df = df.copy()
    new_df['TimeLaggedPrecipitation'] = df['dtDate'].map(lambda row: timelagged_precipitation(df,row))
    return new_df


def timelagged_windspeed(df, obs_date, lag_days_LB=21, lag_days_UB=0):
    """
    Calcuates the average windspeed in Chicago in some time window before an observation
    
    Parameters
    ----------
    df: dataframe with weather info.  I hope there is date info as datetime objects
    obs_date: a datetime timestamp for the observation
    lag_days_LB: chronological lower bound of time window (inclusive), in number of days *before* observation
    lag_days_UB: chronological upper bound of time window (inclusive), in number of days *before* observation 

    Returns
    -------
    Average precipitation in time window 
    """
    
    Date_obs = obs_date
    
    time_lag_LB = dt.timedelta(days=lag_days_LB)
    time_lag_UB = dt.timedelta(days=lag_days_UB)
    
    Date_LB = Date_obs - time_lag_LB
    Date_UB = Date_obs - time_lag_UB
    
    window_Vmean = df[(df.dtDate >= Date_LB) & (df.dtDate <= Date_UB)].avg_AvgSpeed.mean()
    
    return window_Vmean


def make_timelagged_windspeed_col(df):
    new_df = df.copy()
    new_df['TimeLaggedWindspeed'] = df['dtDate'].map(lambda row: timelagged_windspeed(df,row))
    return new_df


def make_avg_weather_columns(df):
    """ Calculates average meterological data on a given date.  Average of both 
    weather stations
    
    Parameters
    ----------
    df: dataframe with weather info.  I hope there is date info as datetime objects
    
    
    Returns
    -------
    new_df: copy of 'df' with new columns added
    
    """
    
    new_df = df.copy()
    avg_Tavg_dict = dict(new_df.groupby(by ='Date').Tavg.mean())
    avg_PrecipTotal_dict = dict(new_df.groupby(by ='Date').PrecipTotal.mean())
    avg_AvgSpeed_dict = dict(new_df.groupby(by ='Date').AvgSpeed.mean())

    new_df['avg_Tavg'] = new_df['Date'].map(avg_Tavg_dict)
    new_df['avg_PrecipTotal'] = new_df['Date'].map(avg_PrecipTotal_dict)
    new_df['avg_AvgSpeed'] = new_df['Date'].map(avg_AvgSpeed_dict)
    return new_df
    
    

def add_six_Truslow_cols(df):
    """Calls a sequence of functions to add five engineered-feature columns to a dataframe.
    Input dataframe must include the original columns from the 'weather.csv' dataset, or
    at least a cleaned version with the same column titles, and with all numeric data.
    
    Parameters
    ----------
    df: Dataframe with weather info.  

    Returns
    -------
    Modified version of 'df', with five columns added.
    """
    
    new_df = make_datetimeobject_col(df)
    
    new_df = make_avg_weather_columns(new_df)
        
    new_df = make_timelagged_daylight_col(new_df)
    
    new_df = make_timelagged_temperature_col(new_df)
    
    new_df = make_timelagged_precipitation_col(new_df)
    
    new_df = make_timelagged_windspeed_col(new_df)

    return new_df

In [87]:
weather = add_six_Truslow_cols(weather)

In [88]:
# droping dt date
weather.drop(['dtDate'], axis=1, inplace=True)

In [89]:
weather.head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,Daylight,avg_Tavg,avg_PrecipTotal,avg_AvgSpeed,TimeLaggedDaylight,TimeLaggedTemperature,TimeLaggedPrecipitation,TimeLaggedWindspeed
0,1,2007-05-01,83,50,67.0,14,51,56,0,2,0.0,29.10,29.82,1.7,27,9.2,14,67.5,0.0,9.40,12.376468,67.500000,0.0,9.400000
1,2,2007-05-01,84,52,68.0,999,51,57,0,3,0.0,29.18,29.82,2.7,25,9.6,14,67.5,0.0,9.40,12.376468,67.500000,0.0,9.400000
2,1,2007-05-02,59,42,51.0,-3,42,47,14,0,0.0,29.38,30.09,13.0,4,13.4,14,51.5,0.0,13.40,12.423742,59.500000,0.0,11.400000
3,2,2007-05-02,60,43,52.0,999,42,47,13,0,0.0,29.44,30.08,13.3,2,13.4,14,51.5,0.0,13.40,12.423742,59.500000,0.0,11.400000
4,1,2007-05-03,66,46,56.0,2,40,48,9,0,0.0,29.39,30.12,11.7,7,11.9,14,57.0,0.0,12.55,12.470968,58.666667,0.0,11.783333


In [90]:
weather_stn1 = weather[weather['Station']==1]
weather_stn2 = weather[weather['Station']==2]
weather_stn1 = weather_stn1.drop('Station', axis=1)
weather_stn2 = weather_stn2.drop('Station', axis=1)
weather = weather_stn1.merge(weather_stn2, on='Date')

In [91]:
weather.columns 

Index(['Date', 'Tmax_x', 'Tmin_x', 'Tavg_x', 'Depart_x', 'DewPoint_x',
       'WetBulb_x', 'Heat_x', 'Cool_x', 'PrecipTotal_x', 'StnPressure_x',
       'SeaLevel_x', 'ResultSpeed_x', 'ResultDir_x', 'AvgSpeed_x',
       'Daylight_x', 'avg_Tavg_x', 'avg_PrecipTotal_x', 'avg_AvgSpeed_x',
       'TimeLaggedDaylight_x', 'TimeLaggedTemperature_x',
       'TimeLaggedPrecipitation_x', 'TimeLaggedWindspeed_x', 'Tmax_y',
       'Tmin_y', 'Tavg_y', 'Depart_y', 'DewPoint_y', 'WetBulb_y', 'Heat_y',
       'Cool_y', 'PrecipTotal_y', 'StnPressure_y', 'SeaLevel_y',
       'ResultSpeed_y', 'ResultDir_y', 'AvgSpeed_y', 'Daylight_y',
       'avg_Tavg_y', 'avg_PrecipTotal_y', 'avg_AvgSpeed_y',
       'TimeLaggedDaylight_y', 'TimeLaggedTemperature_y',
       'TimeLaggedPrecipitation_y', 'TimeLaggedWindspeed_y'],
      dtype='object')

In [92]:
cols = ['TimeLaggedDaylight_y', 'TimeLaggedTemperature_y',
       'TimeLaggedPrecipitation_y', 'TimeLaggedWindspeed_y', 'Depart_y',
       'avg_Tavg_y', 'avg_PrecipTotal_y', 'avg_AvgSpeed_y']

weather.drop(cols, axis=1, inplace=True)

In [93]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1472 entries, 0 to 1471
Data columns (total 37 columns):
Date                         1472 non-null datetime64[ns]
Tmax_x                       1472 non-null int64
Tmin_x                       1472 non-null int64
Tavg_x                       1472 non-null float64
Depart_x                     1472 non-null int64
DewPoint_x                   1472 non-null int64
WetBulb_x                    1472 non-null int64
Heat_x                       1472 non-null int64
Cool_x                       1472 non-null int64
PrecipTotal_x                1472 non-null float64
StnPressure_x                1472 non-null float64
SeaLevel_x                   1472 non-null float64
ResultSpeed_x                1472 non-null float64
ResultDir_x                  1472 non-null int64
AvgSpeed_x                   1472 non-null float64
Daylight_x                   1472 non-null int64
avg_Tavg_x                   1472 non-null float64
avg_PrecipTotal_x            1472 non

In [94]:
weather.to_csv('weather_cleaned_JT_features.csv', index=False)